# Weather Activity Planner

This notebook builds a 3-step workflow:

1. Ask OpenAI to parse the user input into structured data (city + activity preferences).
2. Use that structured response to call a weather API.
3. Send weather results back to OpenAI to generate a friendly plan and recommendations.

**Weather API used:** Open-Meteo (free, no key)
- Geocoding: `https://geocoding-api.open-meteo.com/v1/search`
- Forecast: `https://api.open-meteo.com/v1/forecast`

In [1]:
# If needed, uncomment and run:
# !pip install openai requests python-dotenv

import json
import os
from datetime import datetime
from pathlib import Path

import requests
from dotenv import load_dotenv
from openai import OpenAI

# Load .env from project root (works when run from ImmunePlan or weather_app/)
load_dotenv(Path.cwd() / ".env") or load_dotenv(Path.cwd().parent / ".env")

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Set OPENAI_API_KEY before running this notebook.")

MODEL_NAME = "gpt-5.2"

In [7]:
# User input should include BOTH:
# - the city
# - what kinds of things they want to do today

user_input = (
    "you need to give me the weather forcast for the 28th  for Gig Harbor, WA.  The 28th is my 28th wedding anniversary.  I am looking for some amazing things todo with my wife.  If the weather sunny then guide us to an easy drive, if raning then give use something indores todo.   I like coffee shops, museums, and short walks and exploring nature if the weather is nice. "
)

print(user_input)

you need to give me the weather forcast for the 28th  for Gig Harbor, WA.  The 28th is my 28th wedding anniversary.  I am looking for some amazing things todo with my wife.  If the weather sunny then guide us to an easy drive, if raning then give use something indores todo.   I like coffee shops, museums, and short walks and exploring nature if the weather is nice. 


In [8]:
# Step 1: Use OpenAI tool/function calling to parse structured fields

tools = [
    {
        "type": "function",
        "function": {
            "name": "extract_city_and_preferences",
            "description": "Extract city and activity preferences from user text.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "City name mentioned by the user"
                    },
                    "activity_preferences": {
                        "type": "string",
                        "description": "What kinds of activities they want today"
                    }
                },
                "required": ["city", "activity_preferences"],
                "additionalProperties": False
            }
        }
    }
]

parse_response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {
            "role": "system",
            "content": "Extract city and activity preferences accurately."
        },
        {
            "role": "user",
            "content": user_input
        }
    ],
    tools=tools,
    tool_choice={"type": "function", "function": {"name": "extract_city_and_preferences"}}
)

call = parse_response.choices[0].message.tool_calls[0]
parsed = json.loads(call.function.arguments)

city = parsed["city"]
activity_preferences = parsed["activity_preferences"]

print("Parsed city:", city)
print("Parsed preferences:", activity_preferences)

Parsed city: Gig Harbor, WA
Parsed preferences: Anniversary day plans; if sunny: easy drive + short walks/exploring nature. If raining: indoor activities. Likes coffee shops and museums.


In [9]:
# Step 2: Use parsed city to call Open-Meteo geocoding + forecast APIs

def geocode_city(city_name: str) -> dict:
    geo_url = "https://geocoding-api.open-meteo.com/v1/search"

    # Try multiple query variations (Open-Meteo can be picky with "City, ST" format)
    queries = [
        city_name,
        city_name.split(",")[0].strip(),  # "Austin, TX" -> "Austin"
        city_name.replace(", TX", ", Texas").replace(", tx", ", Texas"),
    ]
    queries = list(dict.fromkeys(queries))  # dedupe while preserving order

    for query in queries:
        params = {
            "name": query,
            "count": 5,
            "language": "en",
            "format": "json"
        }
        resp = requests.get(geo_url, params=params, timeout=20)
        resp.raise_for_status()
        data = resp.json()

        if "results" in data and data["results"]:
            result = data["results"][0]
            break
    else:
        raise ValueError(f"Could not geocode city: {city_name}")

    return {
        "name": result.get("name"),
        "country": result.get("country"),
        "latitude": result.get("latitude"),
        "longitude": result.get("longitude"),
        "timezone": result.get("timezone")
    }


def get_weather(latitude: float, longitude: float, timezone: str = "auto") -> dict:
    weather_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current": "temperature_2m,precipitation,weather_code,wind_speed_10m",
        "daily": "temperature_2m_max,temperature_2m_min,precipitation_probability_max",
        "forecast_days": 1,
        "timezone": timezone
    }
    resp = requests.get(weather_url, params=params, timeout=20)
    resp.raise_for_status()
    return resp.json()

location = geocode_city(city)
weather = get_weather(location["latitude"], location["longitude"], timezone="auto")

weather_snapshot = {
    "location": location,
    "current": weather.get("current", {}),
    "today": {
        "date": weather.get("daily", {}).get("time", [None])[0],
        "temp_max": weather.get("daily", {}).get("temperature_2m_max", [None])[0],
        "temp_min": weather.get("daily", {}).get("temperature_2m_min", [None])[0],
        "precip_prob_max": weather.get("daily", {}).get("precipitation_probability_max", [None])[0],
    },
    "fetched_at": datetime.utcnow().isoformat() + "Z"
}

print(json.dumps(weather_snapshot, indent=2))

{
  "location": {
    "name": "Gig Harbor",
    "country": "United States",
    "latitude": 47.32926,
    "longitude": -122.58013,
    "timezone": "America/Los_Angeles"
  },
  "current": {
    "time": "2026-02-26T20:00",
    "interval": 900,
    "temperature_2m": 7.1,
    "precipitation": 0.0,
    "weather_code": 0,
    "wind_speed_10m": 16.3
  },
  "today": {
    "date": "2026-02-26",
    "temp_max": 11.7,
    "temp_min": 4.2,
    "precip_prob_max": 13
  },
  "fetched_at": "2026-02-27T04:05:50.882344Z"
}


/tmp/ipykernel_808850/879105651.py:66: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "fetched_at": datetime.utcnow().isoformat() + "Z"


In [10]:
# Step 3: Send weather + preferences back to OpenAI for user-facing recommendations

final_response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {
            "role": "system",
            "content": (
                "You are a helpful local planning assistant. "
                "Use weather + user preferences to propose practical activities for today. "
                "Give 4-6 activity ideas and explain why each fits the weather."
            )
        },
        {
            "role": "user",
            "content": (
                f"Original user input: {user_input}\n\n"
                f"Parsed city: {city}\n"
                f"Parsed activity preferences: {activity_preferences}\n\n"
                f"Weather snapshot JSON:\n{json.dumps(weather_snapshot, indent=2)}"
            )
        }
    ],
    temperature=0.7
)

answer = final_response.choices[0].message.content
print(answer)

I can’t reliably give a **forecast for Feb 28** from the weather snapshot you provided—it only includes **current conditions (Feb 26)** and **today’s highs/lows/precip chance (Feb 26)**, not the 28th. If you tell me whether the **28th looks sunny or rainy** (or share a forecast screenshot / hourly summary), I’ll tailor this perfectly.

In the meantime, here are **anniversary-ready options in/near Gig Harbor** with **sunny vs. rainy** branches so you can decide quickly once you see the forecast.

## If it’s SUNNY (easy drive + short walks + nature)
1) **Easy scenic drive: Gig Harbor → Tacoma waterfront loop**
   - **Why it fits sun:** Clear weather makes the water and mountain views pop, and it’s a low-effort, romantic drive.
   - Pair it with a **short stroll** along Ruston Way / waterfront paths (flat, easy, lots of benches).

2) **Fort Worden (Port Townsend) day trip (easy ferry/drive adventure)**
   - **Why it fits sun:** Best enjoyed when it’s bright—bluffs, beaches, and lighthouse